In [1]:
# 导入相关库
import os
import pandas as pd
from tqdm import tqdm,tqdm_notebook
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import accuracy_score
import time
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.model_selection import StratifiedKFold,GroupKFold
from gensim.models import FastText, Word2Vec
import re
from keras.layers import *
from keras.models import *
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import *
from keras.layers.advanced_activations import LeakyReLU, PReLU
import keras.backend as K
from keras.optimizers import *
from keras.utils import to_categorical
import tensorflow as tf
import random as rn
import gc
import logging
import gensim
import jieba
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
set_session(sess)
tqdm.pandas()
os.environ['PYTHONHASHSEED'] = '0'
# 显卡使用（如没显卡需要注释掉）
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
np.random.seed(1024)
rn.seed(1024)
tf.set_random_seed(1024)
path="../data/"
os.listdir("../data/")



Using TensorFlow backend.


['member_info_0926.txt',
 'question_info_0926.txt',
 'word_vectors_64d.txt',
 'invite_info_0926.txt',
 'answer_info_0926.txt',
 'topic_vectors_64d.txt',
 'invite_info_evaluate_2_0926.txt',
 'single_word_vectors_64d.txt',
 '.ipynb_checkpoints',
 'invite_info_evaluate_1_0926.txt',
 'sample_sub_1.txt']

In [2]:
if not os.path.exists("out"):
    os.mkdir("out")
x1_te=np.load("./out/x1_te.npy")
x2_te=np.load("./out/x2_te.npy")
x3_te=np.load("./out/x3_te.npy")
x4_te=np.load("./out/x4_te.npy")
x5_te=np.load("./out/x5_te.npy")
x6_te=np.load("./out/x6_te.npy")
test_fe=np.load("./out/test_fe.npy")

emb1 = np.load("./out/emb1.npy")
emb2 = np.load("./out/emb2.npy")
emb3 = np.load("./out/emb3.npy")
emb4 = np.load("./out/emb4.npy")
emb5 = np.load("./out/emb5.npy")
emb6 = np.load("./out/emb6.npy")

In [8]:
# 需要用到的函数
class AdamW(Optimizer):
    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4,  # decoupled weight decay (1/4)
                 epsilon=1e-8, decay=0., **kwargs):
        super(AdamW, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
            # decoupled weight decay (2/4)
            self.wd = K.variable(weight_decay, name='weight_decay')
        self.epsilon = epsilon
        self.initial_decay = decay

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]
        wd = self.wd  # decoupled weight decay (3/4)

        lr = self.lr
        if self.initial_decay > 0:
            lr *= (1. / (1. + self.decay * K.cast(self.iterations,
                                                  K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1
        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                     (1. - K.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        self.weights = [self.iterations] + ms + vs

        for p, g, m, v in zip(params, grads, ms, vs):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            # decoupled weight decay (4/4)
            p_t = p - lr_t * m_t / (K.sqrt(v_t) + self.epsilon) - lr * wd * p

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'decay': float(K.get_value(self.decay)),
                  'weight_decay': float(K.get_value(self.wd)),
                  'epsilon': self.epsilon}
        base_config = super(AdamW, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


from keras.engine.topology import Layer
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [9]:
from keras.initializers import *

def model_conv(emb1,emb2, emb3,emb4,emb5,emb6):
    
    K.clear_session()

    emb_layer_1 = Embedding(
        input_dim=emb1.shape[0],
        output_dim=emb1.shape[1],
        weights=[emb1],
        input_length=88,
        trainable=False
    )
    emb_layer_2 = Embedding(
        input_dim=emb2.shape[0],
        output_dim=emb2.shape[1],
        weights=[emb2],
        input_length=6,
        trainable=False
    )
    
    emb_layer_3 = Embedding(
        input_dim=emb3.shape[0],
        output_dim=emb3.shape[1],
        weights=[emb3],
        input_length=57,
        trainable=False
    )
    
    emb_layer_4 = Embedding(
        input_dim=emb4.shape[0],
        output_dim=emb4.shape[1],
        weights=[emb4],
        input_length=17,
        trainable=False
    )
    emb_layer_5 = Embedding(
        input_dim=emb5.shape[0],
        output_dim=emb5.shape[1],
        weights=[emb5],
        input_length=5,
        trainable=False
    )
    
    emb_layer_6 = Embedding(
        input_dim=emb6.shape[0],
        output_dim=emb6.shape[1],
        weights=[emb6],
        input_length=17,
        trainable=False
    )
    
    
    
    
    seq1 = Input(shape=(88,))
    seq2 = Input(shape=(6,))
    seq3 = Input(shape=(57,))  
    seq4 = Input(shape=(17,)) 
    seq5 = Input(shape=(5,)) 
    seq6 = Input(shape=(17,)) 
    
    x1 = emb_layer_1(seq1)
    x2 = emb_layer_2(seq2)
    x3 = emb_layer_3(seq3)
    x4 = emb_layer_4(seq4)
    x5 = emb_layer_5(seq5)
    x6 = emb_layer_6(seq6)
    
    sdrop= SpatialDropout1D(rate=0.2)

    x1 = sdrop(x1)
    x2 = sdrop(x2)
    x3 = sdrop(x3)
    x4 = sdrop(x4)
    x5 = sdrop(x5)
    x6 = sdrop(x6)
    
    
    lstm_layer1 = Bidirectional(CuDNNLSTM(64, return_sequences=True, 
kernel_initializer=glorot_uniform(seed = 123)))
    gru_layer1 = Bidirectional(CuDNNGRU(64, return_sequences=True, 
kernel_initializer=glorot_uniform(seed = 123)))
    
    lstm_layer2 = Bidirectional(CuDNNLSTM(64, return_sequences=True, 
kernel_initializer=glorot_uniform(seed = 123)))
    gru_layer2 = Bidirectional(CuDNNGRU(64, return_sequences=True, 
kernel_initializer=glorot_uniform(seed = 123)))
    
    lstm_layer3 = Bidirectional(CuDNNLSTM(64, return_sequences=True, 
kernel_initializer=glorot_uniform(seed = 123)))
    gru_layer3 = Bidirectional(CuDNNGRU(64, return_sequences=True, 
kernel_initializer=glorot_uniform(seed = 123)))
    
    avg_pool = GlobalAveragePooling1D()
    max_pool = GlobalMaxPooling1D()
    
    x1_lstm = lstm_layer1(x1)
    x1_gru = gru_layer1(x1_lstm)
    x1_att = Attention(88)(x1_gru)
    
    x2_lstm = lstm_layer2(x2)
    x2_gru = gru_layer2(x2_lstm)
    x2_att = Attention(6)(x2_gru)
    
    x3_lstm = lstm_layer2(x3)
    x3_gru = gru_layer2(x3_lstm)
    x3_att = Attention(57)(x3_gru)
    
    x4_lstm = lstm_layer2(x4)
    x4_gru = gru_layer2(x4_lstm)
    x4_att = Attention(17)(x4_gru)
    
    x5_lstm = lstm_layer2(x5)
    x5_gru = gru_layer2(x5_lstm)
    x5_att = Attention(5)(x5_gru)
    
    x6_lstm = lstm_layer3(x6)
    x6_gru = gru_layer3(x6_lstm)
    x6_att = Attention(17)(x6_gru)
    
    x1_feature = concatenate([x1_att,avg_pool(x1_gru),max_pool(x1_gru)])
    x2_feature = concatenate([x2_att,avg_pool(x2_gru),max_pool(x2_gru)]) 
    x3_feature = concatenate([x3_att,avg_pool(x3_gru),max_pool(x3_gru)]) 
    x4_feature = concatenate([x4_att,avg_pool(x4_gru),max_pool(x4_gru)]) 
    x5_feature = concatenate([x5_att,avg_pool(x5_gru),max_pool(x5_gru)]) 
    x6_feature = concatenate([x6_att,avg_pool(x6_gru),max_pool(x6_gru)]) 
    
    merge1 = Multiply()([x2_feature, x3_feature])
    merge2 = Multiply()([x2_feature, x5_feature])
    merge3 = Multiply()([x4_feature, x6_feature])
    
#     merge1 = esim(x2,x3)
#     merge2 = esim(x2,x4)
#     merge3 = esim(x2,x5)
    
    hin = Input(shape=(101, ))
    htime = Dense(101, activation='relu')(hin)
    
    x = concatenate([merge1,merge2,merge3,x1_feature,x2_feature,x3_feature,x4_feature,x5_feature,x6_feature, htime])
    
    x = Dense(128,kernel_initializer=he_uniform(seed=123), activation='relu',)(x)
    pred = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=[seq1,seq2,seq3,seq4,seq5,seq6, hin], outputs=pred)
    from keras.utils import multi_gpu_model
#     model = multi_gpu_model(model, 4)
    model.compile(loss='binary_crossentropy',
                  optimizer=AdamW(lr=0.001,weight_decay=0.02,),metrics=["accuracy"])
    return model
gc.collect()

154986

In [20]:
from keras.models import load_model
from keras.utils import CustomObjectScope
test_pred=0
for i in range(5):
    model_path="out/new_final_all_data_self_w2v_nn_chizhu_{}.h5".format(i)
    model = model_conv(emb1, emb2,emb3,emb4,emb5,emb6)
    model.load_weights(model_path)
    if i==0:model.summary()
    test_pred += np.squeeze(model.predict([x1_te,x2_te, x3_te,x4_te, x5_te,x6_te,test_fe],batch_size=2048,verbose=1)/5)
        
        

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 6)            0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 57)           0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 5)            0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 17)           0                                            
__________________________________________________________________________________________________
input_6 (I

In [21]:
sub=pd.read_csv("./out/invite_info_evaluate_2_0926.txt",sep="\t",names=["qid","mid","time"])
sub['label']=test_pred[1141683:]

In [22]:
if not os.path.exists("submit"):
    os.mkdir("submit")
sub.to_csv("nn_testB.txt",sep="\t",index=False,header=None)

In [23]:
sub['label'].mean()

0.13132545351982117